In [1]:
#'''!pip install pymysql'''

In [2]:
import sqlite3
import pymysql
import pandas as pd
import numpy as np

In [3]:
# Conectar-se ao banco de dados SQLite
sqlite_conn = sqlite3.connect('yfhistoric.db')
sqlite_cursor = sqlite_conn.cursor()

# Conectar-se ao banco de dados MySQL
mysql_conn = pymysql.connect(host='192.168.15.57', user='yanadm', password='121212', database='yfhistoric')
mysql_cursor = mysql_conn.cursor()
mysql_cursor.execute("SET foreign_key_checks = 0")

0

In [4]:
# Obter os nomes das tabelas do banco de dados SQLite
tables = sqlite_conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
table_names = [table[0] for table in tables]


# Criar um dicionário para armazenar os DataFrames de cada tabela
yfhistoricdic = {}

# Iterar sobre as tabelas e criar um DataFrame para cada uma
for table_name in table_names:
    # Ler os dados da tabela e criar um DataFrame
    df = pd.read_sql_query(f"SELECT * FROM '{table_name}';", sqlite_conn)
    # Armazenar o DataFrame no dicionário, usando o nome da tabela como chave
    yfhistoricdic[table_name] = df



In [5]:

yfhistoricdic = {key.replace("=", "_COM_"): value for key, value in yfhistoricdic.items()}
yfhistoricdic = {key.replace(".", "_"): value for key, value in yfhistoricdic.items()}
yfhistoricdic = {key.replace("^", "STOCK_"): value for key, value in yfhistoricdic.items()}
dataframes_dict = yfhistoricdic



In [6]:
pandas_to_mysql_types = {
    np.dtype('int64'): 'BIGINT',
    np.dtype('float64'): 'FLOAT',
    np.dtype('datetime64[ns]'): 'DATETIME',
    np.dtype('object'): 'TEXT'
}

In [7]:
# Iterar sobre o dicionário de DataFrames
for table_name, df in dataframes_dict.items():
    # Substituir valores NaN por None em cada DataFrame
    dataframes_dict[table_name] = df.where(pd.notna(df), None)

In [8]:
# Iterar sobre o dicionário de DataFrames
for table_name, df in dataframes_dict.items():
    # Criar a tabela no MySQL
    cursor = mysql_conn.cursor()
    # Substituir valores NaN por NULL
    df.fillna(value=pd.NA, inplace=True)  # Use pd.NA para representar NULL
    # Inferir os tipos de dados das colunas e criar a string de colunas para a criação da tabela
    columns = []
    for col, dtype in df.dtypes.items():
        mysql_type = pandas_to_mysql_types.get(dtype, 'TEXT')
        columns.append(f"`{col}` {mysql_type}")
    columns_str = ', '.join(columns)
    
    create_table_query = f"CREATE TABLE IF NOT EXISTS `{table_name}` ({columns_str})"
    cursor.execute(create_table_query)
    print(f"Tabela '{table_name}' criada no MySQL com sucesso.")

    # Inserir os dados na tabela
    for index, row in df.iterrows():
        placeholders = ', '.join(['%s' for _ in df.columns])
        select_query = f'''SELECT 1 FROM {table_name} WHERE '''
        conditions = []
        row_args = []
        for col, value in zip(df.columns, row):
            if col != 'id':  # Ignorar a coluna 'id'
                conditions.append(f"`{col}` = %s")
                row_args.append(None if isinstance(value, float) and pd.isna(value) else value)  # Substituir NaN por None
        where_clause = ' AND '.join(conditions)
        select_query += where_clause
        cursor.execute(select_query, tuple(row_args))
        existing_row = cursor.fetchone()
        if not existing_row:
            insert_query = f'''INSERT INTO {table_name} ({', '.join(df.columns)}) VALUES ({placeholders})'''
            try:
                cursor.execute(insert_query, tuple(row_args))
            except Exception as e:
                print(insert_query)
                print(f"Ocorreu um erro: {e}")
            pass
        else:
            pass

# Confirmar as alterações no MySQL
mysql_conn.commit()


Tabela 'indicators' criada no MySQL com sucesso.
Tabela 'texts' criada no MySQL com sucesso.
Tabela 'niche' criada no MySQL com sucesso.
Tabela 'currencies' criada no MySQL com sucesso.
Tabela 'tickets' criada no MySQL com sucesso.
Tabela 'commodities' criada no MySQL com sucesso.
Tabela 'cripto' criada no MySQL com sucesso.
Tabela 'news' criada no MySQL com sucesso.
Tabela 'summary' criada no MySQL com sucesso.
Tabela 'usd' criada no MySQL com sucesso.
Tabela 'mlearning' criada no MySQL com sucesso.
Tabela 'gc_com_f' criada no MySQL com sucesso.
Tabela 'si_com_f' criada no MySQL com sucesso.
Tabela 'cl_com_f' criada no MySQL com sucesso.
Tabela 'bz_com_f' criada no MySQL com sucesso.
Tabela 'hg_com_f' criada no MySQL com sucesso.
Tabela 'ng_com_f' criada no MySQL com sucesso.
Tabela 'c_com_f' criada no MySQL com sucesso.
Tabela 'w_com_f' criada no MySQL com sucesso.
Tabela 'kc_com_f' criada no MySQL com sucesso.
Tabela 'sb_com_f' criada no MySQL com sucesso.
Tabela 'ypfd_ba' criada no

KeyboardInterrupt: 

In [ ]:
sqlite_conn.close()
mysql_conn.close()